In [215]:
import pandas as pd
from io import StringIO
import re
import numpy as np
import sys
sys.path.append('C:/Users/aurel/Downloads/HS23/CIR/Automatic-Cart-Unity/Assets/Code/Application/ImageToGraph')
from utils import *
pd.set_option('mode.chained_assignment', None)
from PIL import Image
import os
import glob
from tqdm import tqdm

def elaborate_test(filepath_log:str, filpath_App:str, data_dict:dict, user_id:str, first_test:bool):

    data_dict['user_id'].append(user_id)

    with open(filpath_App, 'r') as file:  
        data = file.read()

    # Read data into a DataFrame
    df_app = pd.read_csv(StringIO(data), sep=' - ', header=None, engine='python')
    df_app.columns = ['Timestamp', 'Data']

    df_app['Timestamp'] = pd.to_datetime(df_app['Timestamp'], format='%d/%m/%Y_%H:%M:%S')

    #Time when world was created
    world_init = df_app.loc[df_app['Data'].str.contains('World Init')]['Timestamp'].max()
    data_dict['world_init'].append(world_init)

    #Get the line where the partecipants submits the food
    max_idx = df_app.loc[df_app['Data'].str.contains('Previous')].index.min()-1 

    #When the items where sent to unity
    items_submission = df_app.iloc[max_idx]['Timestamp']
    data_dict['items_submission'].append(items_submission)
    items_data = df_app.iloc[max_idx]['Data']
    match = re.search(r'products:(.*?)(?=\|)', items_data)
    products_string = match.group(1)
    products_list = products_string.split('&')
    data_dict['products_list'].append(products_list)

    #Get the ideal length
    _,_,ideal_lenght = hamiltonian_path(graph, products_list)
    data_dict['ideal_lenght'].append(ideal_lenght)

    string = df_app.loc[df_app['Data'].str.contains('mode')].iloc[0]['Data']
    test_mode = re.search(r'mode:([^|]+)', string).group(1)
    data_dict['test_mode'].append(test_mode)

    #-----------------------------------------------------------Elaborate Log file

    with open(filepath_log, 'r') as file:
        data = file.read()

    # Read data into a DataFrame
    df = pd.read_csv(StringIO(data), sep=' - ', header=None, engine='python')
    df.columns = ['Timestamp', 'Data']
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y_%H:%M:%S')

    # Split data into different categories
    lidar_df = df[df['Data'].str.startswith('Lidar data')].reset_index(drop=True)
    user_df = df[df['Data'].str.startswith('User data')].reset_index(drop=True)
    product_df = df[df['Data'].str.startswith('Product data')].reset_index(drop=True)
    robot_df = df[df['Data'].str.startswith('Robot data')].reset_index(drop=True)
    path_df = df[df['Data'].str.startswith('Path data')].reset_index(drop=True)

    dataframes = [lidar_df,user_df,product_df,robot_df,path_df]

    for df in dataframes:
        columns = re.findall(r'\|([^:]+):', df.iloc[0,1])
        columns.insert(0,'to_drop') 
        df[columns] = df['Data'].apply(lambda x: re.sub(r'\|.*?:', '|', x).strip('|')).str.split('|', expand=True)  
        df.drop(columns=['to_drop','Data'],inplace=True)

    
    bool_cols = ['userInPath', 'sthBack','detect1', 'detect2', 'detect3', 'detect4', 'detect5','detect1b', 'detect2b', 'detect3b', 'detect4b', 'detect5b']
    lidar_df[bool_cols] = lidar_df[bool_cols].replace({'True': True, 'False': False})


    float32_columns_user = ['userSpeed', 'userRotX', 'userRotY', 'userRotZ', 'userRotW', 'userLocX', 'userLocY', 'userLocZ']
    user_df[float32_columns_user] = user_df[float32_columns_user].replace(',', '.', regex=True).astype('float32')

    float32_columns_robot = ['robotOrientationX', 'robotOrientationY', 'robotOrientationZ', 
                            'robotLocX', 'robotLocY', 'robotLocZ', 'linearVel', 'angularVel']
    robot_df[float32_columns_robot] = robot_df[float32_columns_robot].replace(',', '.', regex=True).astype('float32')
    robot_df['grounded'] = robot_df['grounded'].replace({'True': True, 'False': False})

    product_df[['Count', 'CollCount']] = product_df[['Count', 'CollCount']].astype('int8')



    #Starting position: 50.63, 1.35, 11.73


    #Distance walked by the user
    distance_walked_user = np.sqrt((user_df['userLocX'].diff())**2 + (user_df['userLocZ'].diff())**2).sum()/1.2
    data_dict['distance_walked_user'].append(distance_walked_user)

    #Timestamp of when last item is collected
    timestamp_last_item = product_df.loc[product_df['CollCount']==product_df['CollCount'].max()]['Timestamp'].min()
    data_dict['timestamp_last_item'].append(timestamp_last_item)  

    #Check if all items where collected
    data_dict['collected_all'].append(product_df['CollCount'].max()==len(products_list)) 

    #Timestamp of first movement (Start from when at least one unit away from the starting positions)
    timestamp_first_movement = user_df.loc[(user_df['userLocX']-50.63)**2 + (user_df['userLocZ']-11.73)**2 > 1]['Timestamp'].min()
    data_dict['timestamp_first_movement'].append(timestamp_first_movement) 

    if first_test:
        #Add how much the cart traveled
        distance_traveled_cart = np.sqrt((robot_df['robotLocX'].diff())**2 + (robot_df['robotLocZ'].diff())**2).sum()/1.2
        data_dict['distance_traveled_cart'].append(distance_traveled_cart) 

        #How many collisions where detected
        collision_amount = lidar_df.drop(columns='Timestamp').sum().sum()
        data_dict['collision_amount'].append(collision_amount) 

        num_interactions = len(df_app.loc[df_app['Data'].str.contains('Next Item') | df_app['Data'].str.contains('FollowMe') | df_app['Data'].str.contains('Pause')])
        data_dict['num_interactions'].append(num_interactions)
        

    """
    Relevant informations (Two different CSV for before and after):
    Testing Mode
    User_Id 
    Time of the first movement (Maybe consider out of some bound)
    Time of the last item to be collected
    (- Time to collect all the items/the number of items)
    List of items selected
    Time when the person finished to select the items
    (- How much time it took to select the items/the number of items)
    Time when the map was selected
    How long did the person walk
    - Ideal length of path
    ------------ (For second test)
    Robot Collisions
    How many interactions with the robot (vs. items)
    How much did the robot travel
    """

dir = 'C:/Users/aurel/Downloads/HS23/CIR/Automatic-Cart-Unity/Assets/Code/Application/ImageToGraph/'
imagedir = dir+'Model3.png'
image = np.array(Image.open(imagedir).convert('RGB')) #Remember that it reads row by row
image = image[10:210,10:210,:] #Keep only the superkarmet plan inside the frame  C:\Users\aurel\Downloads\HS23\CIR\ImageToGraph\Model3.png
graph = create_graph(image)


first_dict = {'user_id': [], 'world_init': [], 'items_submission': [], 'products_list': [], 'test_mode': [],
              'distance_walked_user': [],'timestamp_last_item': [], 'timestamp_first_movement': [],
              'ideal_lenght': [], 'collected_all': []} 

second_dict = {'user_id': [], 'world_init': [], 'items_submission': [], 'products_list': [], 'test_mode': [],
              'distance_walked_user': [],'timestamp_last_item': [], 'timestamp_first_movement': [],
              'ideal_lenght': [], 'distance_traveled_cart': [], 'collision_amount': [], 'num_interactions': [],
              'collected_all': []} 


# Get the current directory
current_directory = os.getcwd()

# List all folders in the current directory starting with 'test'
partecipants_folders = [os.path.join(current_directory, folder) for folder in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, folder)) and folder.startswith('test')]


for partecipant in tqdm(partecipants_folders):
    partecipant_id = partecipant.split('\\')[-1].strip('test ')

    #Weird formatting of this 
    if partecipant_id == 'AC2':
        continue

    # Get all txt files inside the folder
    txt_files = glob.glob(os.path.join(partecipant, '*.txt'))
    elaborate_test(txt_files[2], txt_files[0], first_dict, partecipant_id, False)
    elaborate_test(txt_files[3], txt_files[1], second_dict, partecipant_id, True) 


100%|██████████| 13/13 [01:14<00:00,  5.77s/it]

{'user_id': ['AC1', 'AC3', 'AC4', 'AC5', 'AC6', 'AC7', 'AN1', 'AN2', 'LH1', 'LH2', 'LH3', 'LH4'], 'world_init': [Timestamp('2023-12-26 15:50:27'), Timestamp('2023-12-28 18:32:45'), Timestamp('2023-12-28 20:41:55'), Timestamp('2023-12-30 11:38:04'), Timestamp('2023-12-30 18:45:36'), Timestamp('2024-01-01 16:57:47'), Timestamp('2024-01-05 11:05:20'), Timestamp('2024-01-05 11:30:27'), Timestamp('2023-12-28 18:34:41'), Timestamp('2023-12-29 15:42:48'), Timestamp('2023-12-29 16:08:08'), Timestamp('2023-12-29 17:57:04')], 'items_submission': [Timestamp('2023-12-26 15:53:27'), Timestamp('2023-12-28 18:36:24'), Timestamp('2023-12-28 20:44:56'), Timestamp('2023-12-30 11:40:47'), Timestamp('2023-12-30 18:47:11'), Timestamp('2024-01-01 16:59:32'), Timestamp('2024-01-05 11:06:04'), Timestamp('2024-01-05 11:31:39'), Timestamp('2023-12-28 18:36:25'), Timestamp('2023-12-29 15:44:26'), Timestamp('2023-12-29 16:09:53'), Timestamp('2023-12-29 18:00:22')], 'products_list': [['Champagne', 'Fruit smoothies

In [217]:
first_run = pd.DataFrame(first_dict)
second_run = pd.DataFrame(second_dict)
first_run.to_csv('first_runs.csv', index=False)
second_run.to_csv('second_runs.csv', index=False)